In [ ]:
import duckdb
import pandas as pd
import tqdm

In [ ]:
cm = pd.read_parquet('./ncdf.parquet').reset_index()
duck = duckdb.connect(':memory:')

In [ ]:
# duck.sql("CREATE VIEW cm AS SELECT * FROM parquet_scan('cm.parquet');")

In [ ]:
duck.sql("CREATE VIEW itoth AS SELECT * FROM parquet_scan('interpactx.parquet/*.parquet');")

In [ ]:
duck.sql("CREATE TABLE interactions AS SELECT list_sort([CMTE_ID, OTHER_ID]) as pair, COUNT(*) as interaction_count FROM itoth GROUP BY pair ORDER BY interaction_count desc")

In [ ]:
duck.sql('SELECT * FROM interactions LIMIT 10')

In [ ]:
cmrs = cm.to_dict('records')
cmrs[0]

In [ ]:
cmdesc = []
for c in tqdm.tqdm(cmrs):
    desc = f'{c["CMTE_NM"]}\n'
    desc += f' {c["CMTE_CITY"]}'
    desc += f', {c["CMTE_ST"]}\n'
    if c["CONNECTED_ORG_NM"] and c['CONNECTED_ORG_NM'] not in ('UNK', 'N/A'):
        desc += f'({c["CONNECTED_ORG_NM"]})\n'
    if c["CMTE_PTY_AFFILIATION"] and c['CMTE_PTY_AFFILIATION'] not in ('UNK', 'N/A'):
        desc += f', {c["CMTE_PTY_AFFILIATION"]} party\n'
    if c["TRES_NM"] and c['TRES_NM'] not in ('UNK', 'N/A'):
        desc += f', Treasurer {c["TRES_NM"]}\n'
    # top_interactors = ', '.join(duck.sql(f'''
    #     SELECT cmte_nm as other_nm, interaction_count FROM cm JOIN (
    #     SELECT interaction_count, 
    #         list_filter(pair, c -> c != '{c["CMTE_ID"]}')[1] AS other 
    #         FROM interactions WHERE list_contains(pair, '{c["CMTE_ID"]}') ORDER BY interaction_count DESC LIMIT 5
    #     ) ON other = cmte_id ORDER BY interaction_count DESC
    # ''').df()['other_nm'])
    # if top_interactors:
    #     desc += f', Interacts with: {top_interactors}\n'
    cmdesc.append({'CMTE_ID': c['CMTE_ID'], 'desc': desc.lower()})

In [ ]:
cd = pd.DataFrame(cmdesc)
cd

In [ ]:
cm.join(cd[['desc']]).to_parquet('./cmdesc.parquet')